In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
df=pd.read_csv('Feature_Selection.csv')
df.head()

,PriorDefault_t,YearsEmployed,CreditScore,Income,Approved
0,1,3.04,6,560.000000,1
1,1,1.50,0,824.000000,1
2,1,3.75,5,3.000000,1
3,1,1.71,0,322.159652,1
4,1,2.50,0,322.159652,1


In [4]:
#### Independent And Dependent features
X=df.drop('Approved',axis=1)
y=df['Approved']

In [5]:
#### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [6]:
X_train

,PriorDefault_t,YearsEmployed,CreditScore,Income
127,1,3.250,2,610.000000
390,0,0.210,0,322.159652
21,1,7.875,6,1260.000000
463,0,0.000,1,53.000000
338,0,0.750,0,67.000000
...,...,...,...,...
359,0,5.500,0,322.159652
192,1,3.085,6,322.159652
629,0,0.250,0,809.000000
559,1,3.500,6,458.000000


In [7]:
y_train

127    1
390    0
21     1
463    0
338    0
      ..
359    0
192    1
629    0
559    1
684    0
Name: Approved, Length: 551, dtype: int64

In [8]:
#!pip install xgboost==0.90

In [9]:
import xgboost as xgb
xgb_classifier=xgb.XGBClassifier().fit(X_train,y_train)
prediction=xgb_classifier.predict(X_test)

In [10]:
#i am getting this error how to fix this even download the xgboost
# did u mean pip install X
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(f'confusion matrix : {confusion_matrix(y_test,prediction)}')
print(f'Accuracy Score : {accuracy_score(y_test,prediction)}')
print(classification_report(y_test,prediction))

confusion matrix : [[69  6]
 [12 51]]
Accuracy Score : 0.8695652173913043
              precision    recall  f1-score   support

           0       0.85      0.92      0.88        75
           1       0.89      0.81      0.85        63

    accuracy                           0.87       138
   macro avg       0.87      0.86      0.87       138
weighted avg       0.87      0.87      0.87       138



In [11]:
### Manual Hyperparameter Tuning
model=xgb.XGBClassifier(n_estimators=300,learning_rate =0.05,
                             max_depth=5,subsample=0.7,min_child_weight=3).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[65 10]
 [12 51]]
0.8405797101449275
              precision    recall  f1-score   support

           0       0.84      0.87      0.86        75
           1       0.84      0.81      0.82        63

    accuracy                           0.84       138
   macro avg       0.84      0.84      0.84       138
weighted avg       0.84      0.84      0.84       138



### Randomized Search CV

In [12]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Various learning rate parameters
learning_rate = ['0.05','0.1', '0.2','0.3','0.5','0.6']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
#Subssample parameter values
subsample=[0.7,0.6,0.8]
# Minimum child weight parameters
min_child_weight=[3,4,5,6,7]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_depth': max_depth,
               'subsample': subsample,
               'min_child_weight': min_child_weight}

In [13]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
Classifier=xgb.XGBClassifier()

In [14]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
xg_random = RandomizedSearchCV(estimator = Classifier, param_distributions = random_grid,scoring='roc_auc', n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [15]:
xg_random.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3, total=   1.2s
[CV] subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV]  subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3, total=   0.7s
[CV] subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3 
[CV]  subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3, total=   0.7s
[CV] subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3 
[CV]  subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3, total=   0.7s
[CV] subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3 
[CV]  subsample=0.7, n_estimators=1100, min_child_weight=6, max_depth=10, learning_rate=0.3, total=   0.7s
[CV] subsample=0.7, n_estimators=600, min_child_weight=6, max_depth=20, learning_rate=0.3 
[CV]  subsample=0.7, n_estimators=600, min_child_weight=6, max_depth=20, learning_rate=0.3, total=   0.4s
[CV] subsample=0.7, n_estimators=600, min_child_weight=6, max_depth=20, learning_rate=0.3 
[CV]  su

...

[CV]  subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2, total=   0.4s
[CV] subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2 
[CV]  subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2, total=   0.4s
[CV] subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2 
[CV]  subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2, total=   0.4s
[CV] subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2 
[CV]  subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2, total=   0.4s
[CV] subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2 
[CV]  subsample=0.6, n_estimators=700, min_child_weight=7, max_depth=5, learning_rate=0.2, total=   0.4s
[CV] subsample=0.7, n_estimators=100, min_child_weight=4, max_depth=30, learning_rate=0.6 
[CV]  subsample=0.7, n_e

[CV]  subsample=0.8, n_estimators=1100, min_child_weight=5, max_depth=25, learning_rate=0.6, total=   0.7s
[CV] subsample=0.8, n_estimators=1100, min_child_weight=5, max_depth=25, learning_rate=0.6 
[CV]  subsample=0.8, n_estimators=1100, min_child_weight=5, max_depth=25, learning_rate=0.6, total=   0.7s
[CV] subsample=0.8, n_estimators=1100, min_child_weight=5, max_depth=25, learning_rate=0.6 
[CV]  subsample=0.8, n_estimators=1100, min_child_weight=5, max_depth=25, learning_rate=0.6, total=   0.7s
[CV] subsample=0.8, n_estimators=800, min_child_weight=5, max_depth=20, learning_rate=0.2 
[CV]  subsample=0.8, n_estimators=800, min_child_weight=5, max_depth=20, learning_rate=0.2, total=   0.5s
[CV] subsample=0.8, n_estimators=800, min_child_weight=5, max_depth=20, learning_rate=0.2 
[CV]  subsample=0.8, n_estimators=800, min_child_weight=5, max_depth=20, learning_rate=0.2, total=   0.5s
[CV] subsample=0.8, n_estimators=800, min_child_weight=5, max_depth=20, learning_rate=0.2 
[CV]  subs

[CV]  subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3, total=   0.2s
[CV] subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3 
[CV]  subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3, total=   0.3s
[CV] subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3 
[CV]  subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3, total=   0.3s
[CV] subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3 
[CV]  subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3, total=   0.3s
[CV] subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3 
[CV]  subsample=0.7, n_estimators=400, min_child_weight=5, max_depth=25, learning_rate=0.3, total=   0.3s
[CV] subsample=0.8, n_estimators=1100, min_child_weight=4, max_depth=10, learning_rate=0.1 
[CV]  subsampl

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  3.5min finished


RandomizedSearchCV(cv=5, estimator=XGBClassifier(), n_iter=100, n_jobs=1,
                   param_distributions={'learning_rate': ['0.05', '0.1', '0.2',
                                                          '0.3', '0.5', '0.6'],
                                        'max_depth': [5, 10, 15, 20, 25, 30],
                                        'min_child_weight': [3, 4, 5, 6, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200],
                                        'subsample': [0.7, 0.6, 0.8]},
                   random_state=42, scoring='roc_auc', verbose=2)

In [16]:
# # fit the model on the whole dataset
# # make a single prediction
# row = [[0,2,12,70000]]
# yhat = xg_random.predict(row)
# print('Predicted Class: %d' % yhat[0])


In [17]:
xg_random.best_params_

{'subsample': 0.8,
 'n_estimators': 200,
 'min_child_weight': 7,
 'max_depth': 30,
 'learning_rate': '0.05'}

In [18]:
best_random_grid=xg_random.best_estimator_

In [19]:
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[67  8]
 [10 53]]
Accuracy Score 0.8695652173913043
Classification report:               precision    recall  f1-score   support

           0       0.87      0.89      0.88        75
           1       0.87      0.84      0.85        63

    accuracy                           0.87       138
   macro avg       0.87      0.87      0.87       138
weighted avg       0.87      0.87      0.87       138



### Bayesian Optimization

In [20]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [21]:
space = {
    'max_depth' : hp.choice('max_depth', range(5, 30, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(20, 205, 5)),
    'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)}

In [22]:

def objective(space):
    import xgboost as xgb
    model = xgb.XGBClassifier(n_estimators = space['n_estimators'],
                            max_depth = int(space['max_depth']),
                            learning_rate = space['learning_rate'],
                            gamma = space['gamma'],
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                            colsample_bytree = space['colsample_bytree']
                                 )


    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [23]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

100%|███████████████████████████████████████████████| 80/80 [00:29<00:00,  2.75trial/s, best loss: -0.8657002457002457]


{'colsample_bytree': 0.68,
 'gamma': 0.34,
 'learning_rate': 0.05,
 'max_depth': 15,
 'min_child_weight': 5.0,
 'n_estimators': 7,
 'subsample': 0.51}

In [24]:
import xgboost as xgb
trainedforest = xgb.XGBClassifier(n_estimators = best['n_estimators'],
                            max_depth = best['max_depth'],
                            learning_rate = best['learning_rate'],
                            gamma = best['gamma'],
                            min_child_weight = best['min_child_weight'],
                            subsample = best['subsample'],
                            colsample_bytree = best['colsample_bytree']).fit(X_train,y_train)
prediction = trainedforest.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))
acc5 = accuracy_score(y_test,prediction)

[[70  5]
 [13 50]]
0.8695652173913043
              precision    recall  f1-score   support

           0       0.84      0.93      0.89        75
           1       0.91      0.79      0.85        63

    accuracy                           0.87       138
   macro avg       0.88      0.86      0.87       138
weighted avg       0.87      0.87      0.87       138



## tpot

In [25]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Various learning rate parameters
learning_rate = ['0.05','0.1', '0.2','0.3','0.5','0.6']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
#Subssample parameter values
subsample=[0.7,0.6,0.8]
# Minimum child weight parameters
min_child_weight=[3,4,5,6,7]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_depth': max_depth,
               'subsample': subsample,
               'min_child_weight': min_child_weight}

In [26]:
from tpot import TPOTClassifier
from xgboost import XGBClassifier

tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                # config_dict={'xgboost.XGBClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(X_train,y_train)


Generation 1 - Current best internal CV score: 0.8511319157939279
Generation 2 - Current best internal CV score: 0.8511319157939279
Generation 3 - Current best internal CV score: 0.8511319157939279
Generation 4 - Current best internal CV score: 0.8584047392362213
Generation 5 - Current best internal CV score: 0.8584047392362213
Best pipeline: BernoulliNB(GaussianNB(input_matrix), alpha=0.1, fit_prior=True)


TPOTClassifier(cv=4, early_stop=12, generations=5,
               log_file=<ipykernel.iostream.OutStream object at 0x0000020874D3E988>,
               offspring_size=12, population_size=24, scoring='accuracy',
               verbosity=2)

In [27]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)

0.8043478260869565


In [28]:
import pickle

 # save the model to disk
filename = 'finalized_model_xgb.sav'
pickle.dump(xg_random, open(filename, 'wb'))

# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))

In [29]:
# from sklearn.externals import joblib 
  
# # Save the model as a pickle in a file 
# joblib.dump(xg_random, 'xgb.pkl') 
  
# # # Load the model from the file 
# # knn_from_joblib = joblib.load('filename.pkl')  
  

In [30]:
# import numpy as np
# import pickle 
# filename = 'finalized_model_xgb.sav'
# loaded_model = pickle.load(open(filename, 'rb'))
# data = np.array([[0,1,12,70000]])
# my_prediction = loaded_model.predict(data)